In [2]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import altair as alt 
alt.data_transformers.enable('json') # work-around to let Altair handle larger data sets

#import matplotlib.pyplot as plt

DataTransformerRegistry.enable('json')

In [3]:
#
'''
data = pd.read_csv("../data/dpt2020.csv", 
                 sep=';',
                 header=0,
                 names=['gender', 'name', 'year', 'dpt', 'births'],
                 converters={
                          #'gender': d.get,  # lambda x: d[x]
                          'name': str.title
                      })

#cleanning
data = data.loc[(data['name'].str.len() > 1)
                & (data['year'] != 'XXXX')
                & ~data['name'].str.startswith('_')].reset_index(drop=True)

data['year'] = data['year'].astype(np.int64)
data['dpt'] = data['dpt'].astype(np.int64)

data.head()
'''

,gender,name,year,dpt,births
0,1,Aadil,1983,84,3
1,1,Aadil,1992,92,3
2,1,Aahil,2016,95,3
3,1,Aaron,1962,75,3
4,1,Aaron,1976,75,3


In [160]:
#
'''
df=pd.DataFrame(columns=["name",'year',"births"])

for key, group in data.groupby(['year','name']):
    year = key[0]
    name=key[1]
    total=group.births.sum()
    df.loc[len(df)] = name, year, total

total = {}
for key, group in df.groupby('year'):
    total[key] = group.births.sum()
    
df['freq'] = df.apply(lambda x :  x[2]/total[x[1]], axis=1)
df['births'] = df['births'].astype(np.int64)
df['year'] = df['year'].astype(np.int64)
#df.to_csv("baby_years.csv", sep=';' ,index=False)
del total
df.head()
'''

,name,year,births,freq
0,Aaliyah,2019,179,0.000355
1,Aaron,2019,2436,0.004836
2,Aarush,2019,6,0.000012
3,Abbie,2019,11,0.000022
4,Abby,2019,138,0.000274


## Chargement des données aggregées

In [4]:
# loading new dataframe clean and tunned 
df=pd.read_csv("../data/baby_years.csv", sep=';')
df.shape

(249113, 4)

##### Besoin de definir des dictionnaires de ranking sur All year  --> Non present dans la dataFrame aggregée

In [5]:
# faire un ranking des prenoms sur toutes les années qui sera utile pour le trie "All"
#

d_freq = {}
d_births = {}
for key, gp in df.groupby('name'):
    d_freq[key] = gp.freq.sum()
    d_births[key] = gp.births.sum()
# sort the dict by value then key => if Same value, sort by name!
d_freq=dict(sorted(d_freq.items(), key=lambda x: x[1], reverse=True)) 
d_births=dict(sorted(d_births.items(), key=lambda x: x[1], reverse=True))


In [306]:
#
'''
from ipywidgets import interact#, interactive, fixed, interact_manual

def f(a, b, c):
    print(a, b,c)
    
    return a, b, c

#interact(f,a=selection, b=year, c=name);

interactive(children=(Dropdown(description='Display :', index=1, options=(None, 'Top 5', 'Top 10', 'Bottom 5',…

## Fonction de synthèse 
Graphique interactive

In [8]:
# Widgets
selection=widgets.Dropdown(
    options=[None, 'Top 5', 'Top 10', 'Bottom 5', 'Bottom 10'],
    value= None,
    description='Display :',
)
year=widgets.Dropdown(
    options=[None,'All'] + sorted(list(df.year.unique())),
    value= None,
    description='Year:',
)
name = widgets.Dropdown(
    options=[None] + sorted(list(df.name.unique())),
    value=None,
    description='Name:',
)

# Drawing function
def make_chart(noms):  # -> penser à inserer traitement data dans la fonction 
    
    donnee=df[df.name.isin(noms)] # data
    selector = alt.selection_multi(fields=['name']) # filter
    # chart
    chart=alt.Chart(donnee,
             width=800,
             height=400).mark_line().encode(
                                x='year:Q',
                                y='freq:Q',
                                tooltip=['name','year','births','freq'],
                                #color = alt.Color('name:N')
                                color=alt.condition(selector, 'name:N', alt.value('lightgray'))
                            ).add_selection(selector)

    
    # Creation de l'histogram
    histo = alt.Chart(donnee, width=800, height=200).mark_bar().encode(
        x='year:Q',#alt.X('births:Q', bin=alt.Bin(maxbins=100)),
        y='births:Q',
        color='name:N'
    ).transform_filter(selector)

    return chart & histo



def visualisation(selection,annee,nom):
    # get value form selectors
    if not (selection==annee==nom): # All selector are not None=>
        if selection:
            select = selection.split(' ')
            select[1] = int(select[1])
        else:
            select = [None, None]

        # update dataframe according to filter
        if nom is None:
            if annee !='All' and select[0]=='Top':
                l_noms=df[df.year==annee].nlargest(select[1],'births', keep='all').name.values
            if annee !='All' and select[0]=='Bottom':
                l_noms=df[df.year==annee].nsmallest(select[1],'births', keep='first').name.values
            if annee =='All' and select[0]=='Top':
                l_noms=list(d_freq.keys())[:5]
            if annee =='All' and select[0]=='Bottom':
                l_noms=list(d_freq.keys())[-5:]
            if annee =='All' and select[0] is None:
                l_noms = list(df.name.unique())
        else:
            l_noms = [nom]

        #data=df[df.name.isin(l_noms)] # data
        #brush = alt.selection_multi(fields=['name']) # filter
        display(make_chart(l_noms))
    
    return None

#Running graph
box = widgets.HBox([selection, year, name]) 
out = widgets.interactive_output(visualisation, {'selection':selection, 'annee':year, 'nom':name});
display(box,out)

Output()

## Que du brouillon ci dessous -> A supprimer
Les fonctions detaillées

In [ ]:
# Widgets
selection=widgets.Dropdown(
    options=[None, 'Top 5', 'Top 10', 'Bottom 5', 'Bottom 10'],
    value= None,
    description='Display :',
)
year=widgets.Dropdown(
    options=[None,'All'] + sorted(list(df.year.unique())),
    value= None,
    description='Year:',
)
name = widgets.Dropdown(
    options=[None] + sorted(list(df.name.unique())),
    value=None,
    description='Name:',
)

# Drawing function
def make_chart(noms):  # -> penser à inserer traitement data dans la fonction 
    # data selection
    donnee=df[df.name.isin(noms)] # data
    selector = alt.selection_multi(fields=['name:N']) # filter
    
    # chart
    chart=alt.Chart(donnee,
             width=800,
             height=400).mark_line().encode(
                                x='year:Q',
                                y='births:Q',
                                tooltip=['name:N','year:Q','births:Q','freq:Q'],
                                #color = alt.Color('name:N')
                                color=alt.condition(selector, 'name:N', alt.value('lightgray'))
                            ).add_selection(selector)
    # histo diversite prenoms/an
    diversity = alt.Chart(df, width=800, height=200
                         ).mark_bar().encode(x='year:Q',
                                            y=alt.Y('nbNameYear:Q', 
                                                    axis=alt.Axis(title='Name diversity per year', 
                                                                  titleColor='#57A44C')),
                                             tooltip=['year',"nbNameYear:Q"]
                                            ).transform_aggregate(nbNameYear='count(name)',
                                                                  groupby=["year"]
                                                                    )
    # nb naissance par an
    annualbirth = alt.Chart(df, width=800, height=200
                           ).mark_line(color='red').encode(x='year:Q',
                                                            y=alt.Y('birthsYear:Q', 
                                                                    axis=alt.Axis(title='sum births per year', 
                                                                                  titleColor='red')),
                                                           tooltip=['year',"birthsYear:Q"]
                                                            ).transform_aggregate(
                                                                                birthsYear='sum(births)',
                                                                                groupby=["year"]
                                                                                )
    # creation d'un plot communmix des
    down = alt.layer(diversity, annualbirth).resolve_scale(y = 'independent')
    # Creation de l'histogram
    histo = alt.Chart(donnee, width=800, height=200).mark_bar().encode(
        x=alt.X('births:Q', bin=alt.Bin(maxbins=100)),
        y='births:Q',
        color='name:N'
    ).transform_filter(selector)

    return chart & histo & down



def visualisation(selection,annee,nom):
    # get value form selectors
    if not (selection==annee==nom): # All selector are not None=>
        if selection:
            select = selection.split(' ')
            select[1] = int(select[1])
        else:
            select = [None, None]

        # update dataframe according to filter
        if nom is None:
            if annee !='All' and select[0]=='Top':
                l_noms=df[df.year==annee].nlargest(select[1],'births', keep='all').name.values
            if annee !='All' and select[0]=='Bottom':
                l_noms=df[df.year==annee].nsmallest(select[1],'births', keep='first').name.values
            if annee =='All' and select[0]=='Top':
                l_noms=list(d_freq.keys())[:5]
            if annee =='All' and select[0]=='Bottom':
                l_noms=list(d_freq.keys())[-5:]
            if annee =='All' and select[0] is None:
                l_noms = list(df.name.unique())
        else:
            l_noms = [nom]

        #data=df[df.name.isin(l_noms)] # data
        #brush = alt.selection_multi(fields=['name']) # filter
        display(make_chart(l_noms))
    
    return None

#Running graph
box = widgets.HBox([selection, year, name]) 
out = widgets.interactive_output(visualisation, {'selection':selection, 'annee':year, 'nom':name});
display(box,out)

In [11]:
# Histogramme des diversité de prenoms par an!
a =alt.Chart(df,
             width=800,
             height=400).mark_bar().encode(x='year:Q',
                                            y=alt.Y('nbNameYear:Q', 
                                                    axis=alt.Axis(title='Name diversity per year', 
                                                                  titleColor='#57A44C')),             
                        ).transform_aggregate(
                                            nbNameYear='count(name)',
                                            groupby=["year"]
                                            )

In [13]:
# Evolution du nombre de naissances par an!
b = alt.Chart(df,
             width=800,
             height=400).mark_line(color='red').encode(x='year:Q',
                                            y=alt.Y('birthsYear:Q', 
                                                    axis=alt.Axis(title='sum births per year', 
                                                                  titleColor='red')),
                        ).transform_aggregate(
                                            birthsYear='sum(births)',
                                            groupby=["year"]
                                            )
alt.layer(a, b).resolve_scale(
    y = 'independent'
)
a & b

alt.VConcatChart(...)

In [286]:
# getting widgets return values

if selection.value:
    select = selection.value.split(' ')
    select[1] = int(select[1])
if year.value:
    annee = year.value
if name.value:
    nom = name.value
print(select, annee, nom)

# update dataframe according to filter
if nom is None:
    if annee !='All' and select[0]=='Top':
        l_noms=df[df.year==annee].nlargest(select[1],'births', keep='all').name.values
    if annee !='All' and select[0]=='Bottom':
        l_noms=df[df.year==annee].nsmallest(select[1],'births', keep='all').name.values
    if annee =='All' and select[0]=='Top':
        l_noms=list(d_freq.keys())[:5]
    if annee =='All' and select[0]=='Bottom':
        l_noms=list(d_freq.keys())[-5:]
else:
    l_noms = [nom]
print(l_noms)

data=df[df.name.isin(l_noms)]

# Altair display

def make_chart(data,selector):  # -> penser à inserer traitement data dans la fonction 
    
    
    chart=alt.Chart(data,
             width=800,
             height=400).mark_line().encode(
                                x='year:Q',
                                y='births:Q',
                                tooltip=['name','year','births','freq'],
                                #color = alt.Color('name:N')
                                color=alt.condition(selector, 'name:N', alt.value('lightgray'))
                            ).add_selection(selector)


    #pop_selection = alt.selection_interval(encodings=['x'])

    # Creation de l'histogram
    histo = alt.Chart(data, width=800, height=200).mark_bar().encode(
        x=alt.X('births:Q', bin=alt.Bin(maxbins=200)),
        y='births:Q',
        color='name:N'
    ).transform_filter(selector)

    return chart & histo

# selection
brush = alt.selection_multi(fields=['name'])

#draw
make_chart(data, brush)

['Top', 5] All None
['Marie', 'Jean', 'Pierre', 'André', 'Michel']


alt.VConcatChart(...)